# nbdev callbacks

> The build docs and test nbs functions are extensible, with callbacks.

The goal of nbdev callbacks: Make it possible for you to customise the testing and build doc processes to suit the needs of your projects, without needing changes to nbdev.

If you'd like to use callbacks, create `nbdev_callbacks.py` in your project. Feel free to copy [nbdev_callbacks.py](https://github.com/fastai/nbdev/blob/master/nbdev_callbacks.py) to get started.

Before we get stuck into how you can use callbacks, we need to explain how *your* `nbdev_callbacks.py` gets imported.

In [ ]:
from nbdev import *
from nbdev.imports import *
show_doc(call_cb)

<h4 id="call_cb" class="doc_header"><code>call_cb</code><a href="https://github.com/fastai/nbdev/tree/master/nbdev/imports.py#L146" class="source_link" style="float:right">[source]</a></h4>

> <code>call_cb</code>(**`cb_name`**, **\*`args`**)

Calls `cb_name` from the `nbdev_callbacks` module but won't fail if it doesn't exist

After making a temporary modification to `sys.path`, this function loads the `nbdev_callbacks` module via a regular python `import`.

To be sure this import reads the `nbdev_callbacks.py` file from your project, `call_cb` will make the parameter `callbacks_path` (see `settings.ini`) the only entry in `sys.path` (defaulting to the project root).

`nbdev_callbacks.py` does not have to exist and doesn't have to contain definitions for all callback handlers.

In [ ]:
test_eq(call_cb('begin_test_nb', 'arg1', 2, 3), 'arg1')
test_eq(call_cb('after_test_nb', 'file.name'), None)
# If we pass an invalid callback name, we get the 1st arg back ...
test_eq(call_cb('bad_cb_name', 'arg1'), 'arg1')
test_eq(call_cb('bad_cb_name', False), False)
test_eq(call_cb('bad_cb_name', 'arg1', 2, 3), 'arg1')
# ... or None if we pass no args. Either way we see no errors
test_eq(call_cb('bad_cb_name'), None)

try:
    call_cb('begin_test_nb', 'arg1')
    assert False, 'An error should be raised as we passed the wrong number of arguments to the cb'
except TypeError: pass

import nbdev_callbacks
nbdev_callbacks.raise_err = lambda: 1/0
try:
    call_cb('raise_err')
    assert False, 'An error should be raised because the cb handler raised an error'
except ZeroDivisionError: pass

If you ever need to import the `nbdev_callbacks` in your code, please make a call to `call_cb` first.

In [ ]:
call_cb('This makes sure nbdev_callbacks is loaded from the right place')
import nbdev_callbacks

In [ ]:
%nbdev_show_doc nbdev_callbacks.begin_test_nb

<h4 id="nbdev_callbacks.begin_test_nb" class="doc_header"><code>nbdev_callbacks.begin_test_nb</code><a href="nbdev_callbacks.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_callbacks.begin_test_nb</code>(**`nb`**, **`file_name`**, **`flags`**)

Called before testing a notebook. Return the notebook to be tested

It is expected that you'll want to modify and return `nb` but your callback handler *could* create and return a new notebook.

`file_name` can't be modified as it has already been used to create `nb`.

`test_nb` calls `begin_test_nb` before checking flags so you *could* use callbacks to modify flags before testing starts.

In [ ]:
%nbdev_show_doc nbdev_callbacks.after_test_nb

<h4 id="nbdev_callbacks.after_test_nb" class="doc_header"><code>nbdev_callbacks.after_test_nb</code><a href="nbdev_callbacks.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_callbacks.after_test_nb</code>(**`file_name`**)

Called after testing a notebook

In [ ]:
%nbdev_show_doc nbdev_callbacks.begin_doc_nb

<h4 id="nbdev_callbacks.begin_doc_nb" class="doc_header"><code>nbdev_callbacks.begin_doc_nb</code><a href="nbdev_callbacks.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_callbacks.begin_doc_nb</code>(**`nb`**, **`file_name`**, **`output_type`**)

Called before converting a notebook to documentation. Return the notebook to be converted

It is expected that you'll want to modify and return `nb` but your callback handler *could* create and return a new notebook.

`file_name` can't be modified as it has already been used to create `nb`.

- When building HTML docs, `convert_nb` will set `output_type='html'`
- When building markdown docs, `convert_md` will set `output_type='md'`

In [ ]:
%nbdev_show_doc nbdev_callbacks.after_doc_nb_preprocess

<h4 id="nbdev_callbacks.after_doc_nb_preprocess" class="doc_header"><code>nbdev_callbacks.after_doc_nb_preprocess</code><a href="nbdev_callbacks.py#L15" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_callbacks.after_doc_nb_preprocess</code>(**`nb`**, **`file_name`**, **`output_type`**)

Called after pre-processing a notebook, before converting to documentation. Return the notebook to be converted

This callback handler gets the same arguments as `begin_doc_nb` but `nb` will have been modifed by processors that do things like:
- add `show_doc` calls for for exported functions and classes
- remove empty cells
- copy images and modify image paths
- add collapse metadata etc

In [ ]:
%nbdev_show_doc nbdev_callbacks.after_doc_nb

<h4 id="nbdev_callbacks.after_doc_nb" class="doc_header"><code>nbdev_callbacks.after_doc_nb</code><a href="nbdev_callbacks.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>nbdev_callbacks.after_doc_nb</code>(**`file_name`**, **`output_type`**)

Called after converting a notebook to documentation

# What kind of things can you do with these callbacks?

## Move img tags to the start of a new line

To display images side-by-side we could use some HTML like:
```
<figure>
    <div style="display:flex">
        <div style="flex:1">
            <figure>
                <img src="images/post_004/02.jpeg">
                <figcaption><center>Temp caption 1</center></figcaption>
            </figure>
        </div>
        <div style="flex:1">
            <figure>
                <img src="images/post_004/03.jpeg">
                <figcaption><center>Temp caption 2</center></figcaption>
            </figure>
        </div>
    </div>
    <figcaption><center>Main caption</center></figcaption>
</figure>
```

Then you find that nbdev will only copy images and modify image paths for zero indented img tags.

You could modify your HTML ... or you could move image tags with a callback:
```python
def begin_doc_nb(nb,file_name,output_type):
    "Called before converting a notebook to documentation. Return the notebook to be converted"
    for cell in nb['cells']:
        if cell['cell_type']=='markdown' and not cell['source'].startswith('`'):
            cell['source']=cell['source'].replace("<img ", "\n<img ")
    return nb
```

## Count the number of tests cells

To output the number of tests that have been run, you could count each cell that is not exported to your library:

```python
from nbdev.export import find_default_export,is_export

def begin_test_nb(nb,file_name,flags):
    test_count=0
    default_export=find_default_export(nb['cells'])
    exports=[is_export(c, default_export) for c in nb['cells']]
    cells=[(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if c['cell_type']=='code']
    for i,c,e in cells:
        if not e: test_count+=1
    print(file_name,'has',test_count,'test cells')
    return nb
```

The above is just one interpretation of "the number of tests". We could easily write callback handlers to count tests in different ways, like:
- the number of `assert` or `test_eq` calls in the notebook
- the number of cells that contain an `assert` or `test_eq` call

# Reserved callback handler names

| Callback handler |                | Call from          | Comments                                         |
|------------------|----------------|--------------------|--------------------------------------------------|
| begin_test_nbs   | after_test_nbs | cli#nbdev_test_nbs | Might be useful if writing test metrics to log files etc |
| begin_doc_nbs  | after_doc_nbs  | cli#nbdev_build_docs |                                                  |